In [51]:
import cv2
import torch
import numpy as np
from PIL import Image


In [52]:
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")

use_large_model = True

if use_large_model:
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
else:
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")

device = "cpu"
midas.to(device)

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if use_large_model:
    transform = midas_transforms.default_transform
else:
    transform = midas_transforms.small_transform


Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/kevinstoesser/.cache/torch/hub/facebookresearch_WSL-Images_main
Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /Users/kevinstoesser/.cache/torch/hub/facebookresearch_WSL-Images_main


Loading weights:  None


Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master


In [53]:
def depth(img):
    cv_image = np.array(img)
    img = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype('uint8')
    img = Image.fromarray(formatted)
    return img


In [54]:
image_path = '/Users/kevinstoesser/Pictures/Background/DSCF1321.jpg'
test_img = Image.open(image_path)
test_depth = depth(test_img)
test_depth = np.array(test_depth)


In [55]:
resize = False

# Read in the image and depth map
image = cv2.imread(image_path)
depth_map = test_depth

# resize for faster test inference
if resize:
    image = cv2.resize(image, [int(image.shape[1]/10), int(image.shape[0]/10)])
    depth_map = cv2.resize(depth_map,  [int(depth_map.shape[1]/10), int(depth_map.shape[0]/10)])
    print('image was resized')

# Convert the depth map to a floating point numpy array with values between 0 and 1
depth_map = depth_map / 255.0

# Set the target depth value (pixels further away from this value will receive a stronger blur)
target_depth = 0.5

# Create an empty output image of the same size as the input
output = np.zeros_like(image)

# Calculate the kernel sizes for all pixels based on the difference between their depth values and the target depth
kernel_sizes = (np.abs(depth_map - target_depth) * 10 + 1).astype(int)
kernel_sizes[kernel_sizes % 2 == 0] += 1  # kernel size must be odd

# Create an empty output image of the same size as the input
output = np.zeros_like(image)

# Loop over each pixel in the image
for y in range(image.shape[0]):
    for x in range(image.shape[1]):
        # Get the kernel size for the current pixel
        kernel_size = kernel_sizes[y, x]

        # Apply a Gaussian blur to the image using the calculated kernel size
        blur = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

        # Set the output pixel to the blurred value
        output[y, x] = blur[y, x]

# Save the output image
cv2.imwrite("output.jpg", output)


/var/folders/s8/q4h_ts2d62d5hvg2074qz35c0000gn/T/ipykernel_79964/3144988765.py:23: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kernel_sizes = (np.abs(depth_map - target_depth) * 10 + 1).astype(np.int)
